In [ ]:
from petastorm import make_reader
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis,skew
from scipy.signal import periodogram
import scipy.signal
from ecgdetectors import Detectors
import matplotlib.ticker as ticker
import pywt
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score,recall_score
from matplotlib.widgets import TextBox, Button
import sys
import pandas as pd
import seaborn as sn
import os
from math import fsum
sys.path.append(os.path.join(os.getcwd(), ".."))
from shared_utils import Time_series_dimensions_calculus as TSD
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
path_csv_ref_label = "/workspaces/maitrise/data/20221006_physio_quality/set-a/REFERENCE.csv"

In [ ]:
###Data organization : Training dataset and Testing dataset + Reference labels

with make_reader(path_petastorm) as reader:
    for sample in reader:
        data = sample
        break

print(data)


In [ ]:
##Get true label
label_ref = pd.read_csv(path_csv_ref_label)
label_ref = label_ref.to_numpy()
Y = label_ref[:,1].copy()
Y_true = Y[Y.copy()!="unlabeled"]
X_true = label_ref[:,0].copy()
X_true = X_true[Y!="unlabeled"].astype(int)


In [ ]:
##Observation of one patients :

def plot_ECG_signal(signal,name,length= data.signal_length,fs = data.sampling_frequency):
     x = np.array(range(0,(len(signal))))
     x = x/fs    
     fig,ax = plt.subplots(nrows = 1,ncols = 2, figsize = (20,10))
     ax[0].plot(x,signal)
     ax[0].set_title(f"Full signal of Lead {name.decode('utf8')}")
     ax[0].grid()
     ax[1].plot(x,signal)
     ax[1].set_title(f"Close up signal of Lead {name.decode('utf8')}")
     ax[1].grid()
     if len(x) == data.signal_length:
          ax[1].set_xlim([0,3])
     else :
          ax[1].set_xlim([0,x[-1]])
     plt.show()

ECG_signal = data.signal
ECG_lead = data.signal_names
fs = data.sampling_frequency
status = data.noun_id
dico_ECG = {}

for i,j in zip(ECG_lead,range(12)):
     dico_ECG[i] = ECG_signal[:,j]
     print(dico_ECG[i].shape)
     plot_ECG_signal(dico_ECG[i],i)
     
     

In [ ]:
###Some utilitary functions : 


def get_time_axis(sign_length,fs):
    x = np.linspace(0,int(sign_length/fs),sign_length)
    return x

def SDR_Quality_lead(SDR_dict_lead,name_lead):
    SDR_good_quality = {}
    SDR_medium_quality = {}
    SDR_bad_quality = {}
    for i in name_lead:
        if (SDR_dict_lead[i][0]<0.5 or SDR_dict_lead[i][0]>0.8):
            SDR_bad_quality[i] = SDR_dict_lead[i]
        elif (SDR_dict_lead[i][0]<0.6 and SDR_dict_lead[i][0]>0.5) or (SDR_dict_lead[i][0]<0.8 and SDR_dict_lead[i][0]>0.7):
            SDR_medium_quality[i] = SDR_dict_lead[i]
        else : 
            SDR_good_quality[i] = SDR_dict_lead[i]
    return SDR_good_quality,SDR_medium_quality,SDR_bad_quality


def wPMF_Quality_lead(wPMF_dict_lead,name_lead):
    wPMF_good_quality = {}
    wPMF_medium_quality = {}
    wPMF_bad_quality = {}
    for i in name_lead:
        if (wPMF_dict_lead[i][0]<0.25 or wPMF_dict_lead[i][0]>0):
            wPMF_bad_quality[i] = wPMF_dict_lead[i]
        elif (wPMF_dict_lead[i][0]<0.5 and wPMF_dict_lead[i][0]>0.25):
            wPMF_medium_quality[i] = wPMF_dict_lead[i]
        elif (wPMF_dict_lead[i][0]>0.5): 
            wPMF_good_quality[i] = wPMF_dict_lead[i]
    return wPMF_good_quality,wPMF_medium_quality,wPMF_bad_quality

def TSD_Quality_lead(TSD_dict_lead,name_lead):
    TSD_good_quality = {}
    TSD_medium_quality = {}
    TSD_bad_quality = {}
    for i in name_lead:
        if (TSD_dict_lead[i][0]>1.40):
            TSD_bad_quality[i] = TSD_dict_lead[i]
        elif (TSD_dict_lead[i][0]<1.40 and TSD_dict_lead[i][0]>1.25):
            TSD_medium_quality[i] = TSD_dict_lead[i]
        elif (TSD_dict_lead[i][0]<1.25): 
            TSD_good_quality[i] = TSD_dict_lead[i]
    return TSD_good_quality,TSD_medium_quality,TSD_bad_quality

def SNR_Quality_lead(SNR_dict_lead,name_lead):
    SNR_good_quality = {}
    SNR_medium_quality = {}
    SNR_bad_quality = {}
    for i in name_lead:
        if (SNR_dict_lead[i][0]<0.5):
            SNR_bad_quality[i] = SNR_dict_lead[i]
        elif (SNR_dict_lead[i][0]>0.5 and SNR_dict_lead[i][0]<10):
            SNR_medium_quality[i] = SNR_dict_lead[i]
        elif (SNR_dict_lead[i][0]>10): 
            SNR_good_quality[i] = SNR_dict_lead[i]
    return SNR_good_quality,SNR_medium_quality,SNR_bad_quality

def HR_quality_lead(HR_dict,name_lead):
    HR_good_quality = {}
    HR_pathological_lead = {}
    HR_bad_quality = {}
    for i in name_lead : 
        val_HR = np.mean(HR_dict[i][0])
        if (val_HR>24 and val_HR<=220):
            HR_good_quality[i] = HR_dict[i]
        elif (val_HR>220 and val_HR<450):
            HR_pathological_lead[i] = HR_dict[i]
        else : 
            HR_bad_quality[i] = HR_dict[i]
    return HR_good_quality,HR_pathological_lead,HR_bad_quality

def set_classification_status(func_name,index_score):
    if func_name == "SDR":
        return SDR_classification_status(index_score)
    elif func_name == "wPMF":
        return wPMF_classification_status(index_score)
    elif func_name == "TSD":
        return TSD_classification_status(index_score)
    elif func_name == "SNR":
        return SNR_classification_status(index_score)
    elif func_name == "HR":
        return HR_classification_status(index_score)

def set_quality_lead(func_name,funct_dict_lead,name_lead):
    if func_name == "SDR":
        return SDR_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "wPMF":
        return wPMF_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "TSD":
        return TSD_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "SNR":
        return SNR_Quality_lead(funct_dict_lead,name_lead)
    elif func_name == "HR":
        return HR_quality_lead(funct_dict_lead,name_lead)

def wPMF_classification_status(mean_wPMF):
    if (mean_wPMF>0.5):
        return "acceptable"
    else : 
        return "unacceptable"

def SDR_classification_status(mean_SDR):
    if (mean_SDR>0.5 and mean_SDR<0.8):
        return "acceptable"
    else : 
        return "unacceptable"

def TSD_classification_status(mean_TSD):
    if (mean_TSD<1.25):
        return "acceptable"
    else : 
        return "unacceptable"

def SNR_classification_status(mean_SNR):
    if mean_SNR>0.5:
        return "acceptable"
    else : 
        return "unacceptable"

def HR_classification_status(mean_HR):
    if mean_HR>24 and mean_HR<300:
        return "acceptable"
    else : 
        return "unacceptable"

def Sorter_X_array(X_arr):
    index_sorted = np.argsort(X_arr)
    X_arr_sort = np.sort(X_arr)
    return X_arr_sort,index_sorted

In [ ]:
###Index Creation : SDR 
### The label will be as follow : 0.8>mean(SDR of all lead) > 0.5 = Acceptable;mean(SDR of all lead) <0.5 or >0.8 = Unacceptable
##For each lead, we will return a mor eprecise classification based on the folloying rules
## SDR<0.5 or SDR>0.8 = Bad quality ; 0.6>SDR>0.5 or 0.8>SDR>0.7= Medium quality; 0.7>SDR>0.6 = Good quality

def SDR_score(signals,name_lead,fs):
    ##SDR coeff:
    SDR_lead = {}
    SDR_arr = np.array([])
    for i in name_lead:
        f,PSD = periodogram(signals[i],fs)
        QRS_signal_PSD = np.sum(PSD[np.logical_and(f>=5,f<=14)])
        ECG_tot = np.sum(PSD[np.logical_and(f>=5,f<=50)],dtype = np.float64)
        if ECG_tot == 0:
            ECG_tot = np.sum(np.abs(signals[i])**2)
            if ECG_tot ==0:
                ECG_tot = 2**63-1
        SDR_val = QRS_signal_PSD/ECG_tot
        SDR_lead[i] = (SDR_val,signals[i])
        SDR_arr = np.append(SDR_arr,SDR_val)
    return SDR_lead,np.mean(SDR_arr)



In [ ]:
###Index Creation : wPMF
### The label will be as follow : mean(SDR of all lead) > 0.5 = Acceptable;mean(SDR of all lead) <0.5 = Unacceptable
##For each lead, we will return a mor eprecise classification based on the folloying rule: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9281614/#B25



def Wavelet_coef(sig,name,lev):
    All_coeff = pywt.wavedec(sig,name,level = lev)

    CA = All_coeff[0]
    CD = All_coeff[1:len(All_coeff)]
    return CA,CD  


def Energy_L2(coeff):
    return np.sum(np.abs(coeff)**2, dtype = np.float64)

def wPMF_score(dico_signal,name_lead,fs):
    waveletname = 'db4'
    level_w = 9
    wPMF_lead = {}
    wPMF_arr = np.array([],dtype = np.float64)
    for i in name_lead:
        CA_w,CD_w = Wavelet_coef(dico_signal[i],waveletname,level_w)
        CD_w = np.array(CD_w,dtype = object)
        CA_w = np.array(CA_w,dtype = object)
        E = np.array([])
        for CD in range(level_w):
            E = np.append(E,Energy_L2(CD_w[-(CD+1)]))
        E = np.append(E,Energy_L2(CA_w[0]))
        Etot = np.sum(E,dtype = np.float64)
        if Etot == 0:
            Etot = Energy_L2(dico_signal[i][:int((2**level_w)-1)])
            if Etot ==0:
                Etot = 2**63-1
        p = E/Etot
        SQI_ECG = np.sum(p[3:6])
        wPMF_lead[i] = (SQI_ECG,dico_signal[i])
        wPMF_arr = np.append(wPMF_arr,SQI_ECG)
    return wPMF_lead, np.mean(wPMF_arr, dtype = np.float64)

In [ ]:
###Index Creation : SNR
### The label will be as follow : mean(SNR of all lead) > 0.5 = Acceptable;mean(SNR of all lead) <0.5 = Unacceptable
##For each lead, we will return a more eprecise classification based on the folloying rules
## SNR<0.5  = Bad quality ; 0.5>SNR<10= Medium quality; SNR>10 = Good quality

def SNR_index(dico_signal,name_lead,fs):
    SNR_lead = {}
    SNR_arr = np.array([],dtype = np.float64)
    for i in name_lead:
        f,PSD = periodogram(dico_signal[i],fs)
        Sig_PSD = np.sum(PSD[np.logical_and(f>2,f<=40)])
        LF_PSD = np.sum(PSD[np.logical_and(f>=0,f<=2)])
        HF_PSD = np.sum(PSD[np.logical_and(f>40,f<=250)])
        if (LF_PSD+HF_PSD == 0.0):
            score = Sig_PSD/(LF_PSD+HF_PSD+0.0001)
        else:
            score = Sig_PSD/(LF_PSD+HF_PSD)
        SNR_lead[i] = (score,dico_signal[i])
        SNR_arr = np.append(SNR_arr,score)
    
    return SNR_lead,np.mean(SNR_arr)
        


In [ ]:
dico_SQI = {"SDR":SDR_score,"wPMF":wPMF_score,"TSD":TSD.TSD_index,"SNR":SNR_index}

In [ ]:
##The general function to run statistical test : 

def Runner_statistic(func,name_func,path_peta,y_true,y):
    matrix = {}
    matrix["Y_True"] = y_true

    ##Dictionary lead quality for each patient with SDR
    lead_patient_history_func = {}

    X_predicted = np.array([])
    Y_predicted = np.array([])
    func_val_index = np.array([])
    with make_reader(path_peta) as reader:
        for sample in reader:
            data = sample
            X_predicted = np.append(X_predicted,int(data.noun_id))
            ECG_signal = data.signal
            ECG_lead = data.signal_names
            fs = data.sampling_frequency
            status = data.noun_id
        
            dico_ECG = {}

            for i,j in zip(ECG_lead,range(12)):
                dico_ECG[i] = ECG_signal[:,j]
            N = len(dico_ECG[ECG_lead[0]])

            func_lead,func_index= func(dico_ECG,ECG_lead,fs)
            prediction = set_classification_status(name_func,func_index)
            func_val_index = np.append(func_val_index,func_index)
            lead_good,lead_medium,lead_bad = set_quality_lead(name_func,func_lead,ECG_lead)

            Y_predicted = np.append(Y_predicted,prediction)
            lead_patient_history_func[status] = np.array([lead_good,lead_medium,lead_bad])

    X_pred_sorted,ind_sort = Sorter_X_array(X_predicted)
    Y_predicted = Y_predicted[ind_sort]
    func_val_index = func_val_index[ind_sort]
    Y_predicted = Y_predicted[y!="unlabeled"]
    func_val_index = func_val_index[y!="unlabeled"]
    matrix["Y_predict"] = Y_predicted
    cm = confusion_matrix(y_true, Y_predicted).ravel()
    if len(cm)>4:
        tp,fn,fp,tn = cm[cm!=0]
    else :
        tp,fn,fp,tn = cm
    print("TP = ",tp)
    print("TN = ",tn)
    print("FP = ",fp)
    print("FN =",fn)
    Acc = (tp+tn)/(tp+tn+fp+fn)
    Prec = tp/(tp+fp)
    Recall = tp/(tp+fn)
    F1 = (2*Recall*Prec)/(Recall+Prec)
    print("Accuracy = ",Acc)
    print("Precision = ",Prec)
    print("Recall = ",Recall)
    print("F1 score = ",F1)

    ##Confusion matrix :
    df = pd.DataFrame(matrix, columns=['Y_True','Y_predict'])
    confusion = pd.crosstab(df['Y_True'], df['Y_predict'], rownames=['Actual'], colnames=['Predicted'],margins = True)
    sn.heatmap(confusion, annot=True,fmt='g')
    plt.title(f"Confusion Matrix for using the {name_func} index")
    plt.show()

    return Y_predicted,func_val_index,lead_patient_history_func

In [ ]:
##Test index SDR
Y_pred_SDR,SDR_val_index,lead_patients_history_SDR = Runner_statistic(SDR_score,"SDR",path_petastorm,Y_true,Y)

In [ ]:
###Test index wPMF

Y_pred_wPMF,wPMF_val_index,lead_patients_history_wPMF =Runner_statistic(wPMF_score,"wPMF",path_petastorm,Y_true,Y)

In [ ]:
###Test index TSD

Y_pred_TSD,TSD_val_index,lead_patients_history_TSD =Runner_statistic(TSD.TSD_index,"TSD",path_petastorm,Y_true,Y)

In [ ]:
##Test index SNR

Y_pred_SNR,SNR_val_index,lead_patients_history_SNR =Runner_statistic(SNR_index,"SNR",path_petastorm,Y_true,Y)

In [ ]:
###Trail 2D dimensions plot : Check if combining 2 index can separate well signal quality assessment

def The_2D_plot_creator(Y1,Y2,correct_label,name_1,name_2,name_label_used = "Original",semilog = False):
    plt.figure()

    for lab,c in zip(["acceptable","unacceptable"],["r","b"]):
        Y1_sel = Y1[correct_label == lab].copy()
        Y2_sel = Y2[correct_label == lab].copy()
        plt.scatter(Y1_sel,Y2_sel,color = c,label = lab,alpha = 0.3)
    plt.legend(["acceptable","unacceptable"])
    plt.grid()
    plt.xlabel(name_1)
    plt.ylabel(name_2)
    if semilog : 
        plt.semilogy()
    plt.title(f"{name_1} VS {name_2} with the {name_label_used} labelisation" )
    plt.show()

###Let's plot!

The_2D_plot_creator(SDR_val_index,wPMF_val_index,Y_true,"SDR","wPMF")
The_2D_plot_creator(SDR_val_index,TSD_val_index,Y_true,"SDR","TSD")
The_2D_plot_creator(wPMF_val_index,TSD_val_index,Y_true,"wPMF","TSD")
The_2D_plot_creator(wPMF_val_index,SNR_val_index,Y_true,"wPMF","SNR",semilog = True)
The_2D_plot_creator(SDR_val_index,SNR_val_index,Y_true,"SDR","SNR",semilog = True)
The_2D_plot_creator(TSD_val_index,SNR_val_index,Y_true,"TSD","SNR",semilog =True)


In [ ]:
###Implementation HeartBeat detector ==> First Step of most SQA method in the litterature
###Goal of this method : Get the RR interval of each metho and check if BPM are in physiological range
### If pathological range ==> acceptable but with a the following message : "Suspicion of pathologie ==> Type of pathology"

###Frequency range considered:
### 24<f<300 : Acceptable (can be pathological but not so sure! need classification for that)
### 300<f<450 : Acceptbable but pathological (BPM with high risk of Fibrillation with multiple focal discharge)
### f>450 or f<24 : Unacceptable

def HR_index_calculator(dico_signal,name_lead,fs):
    detect = Detectors(fs)
    RR_intervals_signal = {}
    mean_RR_interval = np.array([])
    x = get_time_axis(len(dico_signal[name_lead[0]]),fs)
    for i in name_lead:
        r_peaks = detect.pan_tompkins_detector(dico_signal[i])
        r_sec = x[r_peaks]
        r_msec = r_sec*1000
        if len(r_msec) <=1:
            RR_intervals_signal[i] = (0.0,0.0)
            mean_RR_interval = np.append(mean_RR_interval,0.0)
        else:
            RR_bpm_interval = (60/(np.diff(r_msec)))*1000
            RR_intervals_signal[i] = (RR_bpm_interval,dico_signal[i])
            mean_RR_interval = np.append(mean_RR_interval,np.mean(RR_bpm_interval))
        #RR_intervals_signal[i] = (np.diff(r_sec)*1000,r_peaks,np.mean(r_sec))
        #mean_RR_interval = np.append(mean_RR_interval,np.mean(r_sec))
    return RR_intervals_signal,np.mean(mean_RR_interval)




In [ ]:
###Test HR classifiction : 
Y_pred_HR,HR_val_index,lead_patients_history_HR=Runner_statistic(HR_index_calculator,"HR",path_petastorm,Y_true,Y)

In [ ]:
###Bad prediction presentation : Seeing what went wrong with each method

def Derprinterlead(dict_quality,t,prediction,act,quality,patient_ind):
    for i in dict_quality : 
        if type(dict_quality[i][0]) == np.ndarray:
            val = np.mean(dict_quality[i][0])
            plt.figure() 
            plt.plot(t,dict_quality[i][1].copy())
            plt.title(f"Full signal of Lead {i.decode('utf8')} for patient {patient_ind}")
            plt.grid()
            #plt.xlim([0,3])
            plt.figtext(1, 0.7, "HR value = {0:.2f}".format(val))
            plt.figtext( 1, 0.6, "Label assigned = {0:.2f}".format(prediction))
            plt.figtext( 1, 0.5, "Quality = {0:.2f}".format(quality))
        else : 
            plt.figure() 
            plt.plot(t,dict_quality[i][1].copy())
            plt.title(f"Full signal of Lead {i.decode('utf8')} for patient {patient_ind}")
            plt.grid()
            #plt.xlim([0,3])
            plt.figtext(1, 0.7, "Index value = {0:.2f}".format(dict_quality[i][0]))
            plt.figtext(1, 0.6, f"Label assigned = {prediction}")
            plt.figtext(1, 0.5, f"True label = {act}")
            plt.figtext(1, 0.4, "Quality = {}".format(quality))



def bad_prediction_case(pred,actual,X_actual,history_lead_patient,name_lead = ECG_lead[0]):
    index_bad = (actual!=pred)
    patients_concerned = X_actual[index_bad]
    i = patients_concerned[1]
    string = str(i)
    good_quality,medium_quality,bad_quality = history_lead_patient[string.encode()]
    N = 5000
    t = get_time_axis(N,fs)
    Derprinterlead(good_quality,t,pred[np.where(X_actual==i)],actual[np.where(X_actual==i)],"good",i)
    #Derprinterlead(medium_quality,t,pred[np.where(X_actual==i)],actual[np.where(X_actual==i)],"medium",i)
    #Derprinterlead(bad_quality,t,pred[np.where(X_actual==i)],actual[np.where(X_actual==i)],"bad",i)


    
    

In [ ]:
###Test SDR:

bad_prediction_case(Y_pred_TSD,Y_true,X_true,lead_patients_history_TSD)
